In [ ]:
import xml.etree.ElementTree as ET    # XML file parsing module. XML file stores data in a tree form. The main element is the "root"
import numpy as np

def read_in_xml(xml_file, sample, with_stdev = False):   # xml_file = "dsx1.xml"; probably can skip "sample" if there is only 1 replicate

    tree = ET.parse(xml_file)
    root = tree.getroot()    # in this case, root = <data>. 
    # data has one child element (<transcript>, each has a few children, (<sequence> & <reactivity>), stored like an array)
    
    # parsing and storing the data in new variables
    transcript_id = root[0].attrib["id"]    # = "dsx1"
    length = root[0].attrib["length"]       # = "395"
    sequence = root[0][0].text.replace("\t", "").replace("\n", "")    # the RNA sequence into one long string
    reactivity = np.array(root[0][1].text.replace("\t", "").replace("\n", "").split(",")).astype(float)    # reactivity in array form
    ''' reactivity = [0.002]
                     [0.001]
                     [NaN]
                     ...
        row number = 395
        col number = 1
    '''
    
    if with_stdev:    # used when there are more than 1 replicate, probably can be deleted otherwise
        
        stdev = np.array(root[0][2].text.replace("\t", "").replace("\n", "").split(",")).astype(float)
        return {"sample" : sample, 
            "transcript_id" : transcript_id,
            "length" : length,
            "sequence" : sequence, 
            "reactivity" : reactivity,
            "stdev": stdev
           }
    else:
        return {"sample" : sample, 
            "transcript_id" : transcript_id,
            "length" : length,
            "sequence" : sequence, 
            "reactivity" : reactivity
           }
    
    
def convert_xml_to_bpseq(xml_file,outfile):   # input file, output file name

    tmp_data = read_in_xml(xml_file, "")
    
    # just storing reactivity and sequence in new variables
    reactivities = tmp_data["reactivity"]    # 395 x 1 array
    sequence = list(tmp_data["sequence"].replace("T", "U"))    # turning the string sequence into a list of nucleotides and replacing T with U
    
    reactivities = np.nan_to_num(reactivities, nan=-1.0)    # replace NaN to -1.0, non-reactive
    with open(outfile, "w+") as out:                      # opening the output file, "w+" means file is to be edited, the file is now named "out"
        for i in np.arange(1,1+reactivities.shape[0]):   # looping from 1 to 395
            position = int(i)
            line = f"{position} {sequence[position-1]} e1 {reactivities[position-1]}\n"   # basically rewriting the data into a new format
            out.write(line)
            
            ''' how the new format might look like (I haven't tried this but this though)
                1 C 0.002
                2 A 0.001
                3 G -1.0
                4 T -1.0
                ...
            '''
            
#function to help predicting only part of an RNA isoform
#you probably don't need this function. Let me know if you do
def convert_xml_to_bpseq_trimmed(xml_file,outfile, length):

    tmp_data = read_in_xml(xml_file, "")

    reactivities = tmp_data["reactivity"][:length]
    sequence = list(tmp_data["sequence"].replace("T", "U"))[:length]
    
    reactivities = np.nan_to_num(reactivities, nan=-1.0)
    with open(outfile, "w+") as out:
        for i in np.arange(1,1+reactivities.shape[0]):
            position = int(i)
            line = f"{position} {sequence[position-1]} e1 {reactivities[position-1]}\n"
            out.write(line)

In [ ]:
# makes similar assumptions as rf-combine above
#genrates a bp2seq file for use in Eternafold and a csv file for general use

for sample in samples:
    if ("Rep1" in sample):
        combined_sample = "_".join(sample.split("_")[1:])

        for option in ["q22_eq10_ndni"]: #["q22_eq10_ndni", "q22_eq10", "default"]
            for reactive_nt in ["AC", "ACT"]: #["ACGT", "AC", "ACT", "G"]
                for norm_option in [""]: #["", "_raw"]

                    xml_combined = f"{data_folder}/rfcombine/{combined_sample}/{option}_{reactive_nt}{norm_option}/{isoform}.xml"
                    if os.path.isfile(xml_combined):
                        file = read_in_xml(xml_combined, combined_sample)
                        reactivities = file["reactivity"]
                        reactivities = np.nan_to_num(reactivities, nan = -1)
                        reactivity_file = xml_combined.replace(".xml", ".csv")
                        np.savetxt(reactivity_file, reactivities, fmt="%6f")
                        bp_file = xml_combined.replace(".xml", ".bp2seq")
                        convert_xml_to_bpseq(xml_combined, bp_file)

                        #lower part is optional, was used to predict HIV-1 unspliced 5' UTR folding
                        length = 380
                        bp_file = bp_file.replace(".bp2seq", f"_{length}nt.bp2seq")
                        convert_xml_to_bpseq_trimmed(xml_combined,bp_file, length)